In [ ]:
%load_ext autoreload
%autoreload 2
from oxeo.water.datamodules.samplers import RandomSampler, GridSampler
from oxeo.water.datamodules.datasets import TileDataset
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import matplotlib.pyplot as plt
from skimage import img_as_float
from skimage.exposure import equalize_adapthist, rescale_intensity
import numpy as np
from oxeo.water.models.utils import load_tile, TilePath, tile_from_id

In [ ]:
tile_from_id("54_K_10000_34_771")

In [ ]:
tile_path_1 = TilePath(tile_from_id("54_K_10000_34_771"), "landsat-8")
tile_path_2 = TilePath(tile_from_id("54_K_10000_34_772"), "landsat-8") 

In [ ]:
ds = TileDataset([tile_path_1,tile_path_2])
sampler = RandomSampler(ds, 666, 32, 100)#GridSampler(ds, 1000, 100)#

In [ ]:
def worker_init_fn(worker_id):
    """Configures each dataset worker process.

    Just has one job!  To call SatelliteDataset.per_worker_init().
    """
    # get_worker_info() returns information specific to each worker process.
    worker_info = torch.utils.data.get_worker_info()
    if worker_info is None:
        print("worker_info is None!")
    else:
        dataset_obj = worker_info.dataset  # The Dataset copy in this worker process.
        dataset_obj.per_worker_init()


dataloader = DataLoader(ds, batch_size=8, 
                        sampler=sampler, 
                        num_workers=3, 
                        worker_init_fn=worker_init_fn)

In [ ]:
len(dataloader)

In [ ]:
for sample in dataloader:
    print(sample['image'].shape)

In [ ]:
for sample in dataloader:
    img = img_as_float(sample["image"][0,[3,2,1]].numpy().transpose(1,2,0))
    vmin, vmax = np.percentile(img, q=(0, 98))
    img = rescale_intensity(img,in_range=(vmin,vmax))
    plt.imshow(img)
    plt.show()